# Kerakli kutubxonalarni chaqiramiz

In [44]:
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
labeller = LabelEncoder()

# dataset ni yuklaymiz

In [24]:
df = pd.read_csv(r"C:\Users\bunyo\OneDrive\Desktop\AI_Course\ModularProgramProjects\FirstWeekProject\data\raw_data\wind_dataset.csv")

In [25]:
df.sample(6)

,DATE,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G
3490,1970-07-23,4.58,0,11.1,0.0,23.2,0.0,12.3,11.6
5839,1976-12-27,11.67,0,1.8,0.0,4.4,0.0,1.1,-3.5
3558,1970-09-29,13.17,0,2.3,0.0,17.4,0.0,9.8,7.3
5855,1977-01-12,4.96,0,0.4,0.0,2.0,1.0,-2.9,-7.6
3899,1971-09-05,5.04,0,0.0,0.0,18.4,0.0,4.2,3.0
1348,1964-09-10,14.58,0,0.1,0.0,21.8,0.0,16.7,15.6


# DATE ustuni keraksizligi uchun tashab yubaramiz

In [26]:
df = df.drop('DATE', axis=1)

In [28]:
df.isnull().sum()

WIND         0
IND          0
RAIN         0
IND.1       61
T.MAX      621
IND.2       61
T.MIN      674
T.MIN.G    360
dtype: int64

# feature engineering qilish

In [31]:
df['T_AVG'] = (df['T.MAX'] - df['T.MIN']) / 2
# Bu xolatda ortacha xavo xarorati chiqarib olindi

df['IS_RAINIY'] = (df['RAIN'] > 0).astype(int)
# Bu code orqali aniq yomgir yog'ganligi bilib olamiz (1 bolsa yomgir bor. 0 bolsa yomgir yoq)

df['IS_FROST'] = (df['T.MIN.G'] < 0).astype(int)
# Bunda yer muzlagan yoki yoq ligini bilamiz (1 bolsa muzlagan 0 bolsa muzlamagan)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   WIND       6574 non-null   float64
 1   IND        6574 non-null   int64  
 2   RAIN       6574 non-null   float64
 3   IND.1      6513 non-null   float64
 4   T.MAX      5953 non-null   float64
 5   IND.2      6513 non-null   float64
 6   T.MIN      5900 non-null   float64
 7   T.MIN.G    6214 non-null   float64
 8   T_AVG      5898 non-null   float64
 9   IS_RAINIY  6574 non-null   int64  
 10  IS_FROST   6574 non-null   int64  
dtypes: float64(8), int64(3)
memory usage: 565.1 KB


# Class orqli data ga ishlov beramiz

In [41]:
class Preprocessing:
    def __init__(self,df):
        self.df = df
        
# Encoding qiluvchi
    def encoding_qilish(self):
        for col in self.df.columns:
            if self.df[col].dtype == 'object':
                if self.df[col].nunique() <= 3:
                    new_df = pd.get_dummies(self.df[col], prefix='New', dtype=int)
                    self.df.drop(columns=[col],inplace=True)
                    self.df = pd.concat([self.df,new_df],axis=1)
                else:
                    self.df[col] = labeller.fit_transform(self.df[col])
        return self
    

# To'ldruvchi
    def fillingNan(self):
        for cols in self.df.columns:
            if self.df[cols].isnull().any():
                if self.df[cols].dtype == 'object':
                    self.df[cols].fillna(self.df[cols].mode()[0], inplace=True)
                else:
                    self.df[cols].fillna(self.df[cols].mean(),inplace=True)
        return self
    
    
# Scale qiluvchi
    def scaling_qilish(self): 
        for cols in self.df.columns: 
            if self.df[cols].dtype != 'object' and self.df[cols].name != 'WIND': 
                self.df[cols] = minmax.fit_transform(self.df[[cols]]) 
        return self.df


In [42]:
pre_processing = Preprocessing(df)
df1 = pre_processing.fillingNan().encoding_qilish().scaling_qilish()

In [45]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   WIND       6574 non-null   float64
 1   IND        6574 non-null   float64
 2   RAIN       6574 non-null   float64
 3   IND.1      6574 non-null   float64
 4   T.MAX      6574 non-null   float64
 5   IND.2      6574 non-null   float64
 6   T.MIN      6574 non-null   float64
 7   T.MIN.G    6574 non-null   float64
 8   T_AVG      6574 non-null   float64
 9   IS_RAINIY  6574 non-null   float64
 10  IS_FROST   6574 non-null   float64
dtypes: float64(11)
memory usage: 565.1 KB


In [48]:
df1.head()

,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G,T_AVG,IS_RAINIY,IS_FROST
0,13.67,0.0,0.002985,0.0,0.356877,0.00,0.515254,0.443709,0.299465,1.0,1.0
1,11.50,0.0,0.076119,0.0,0.271375,0.00,0.532203,0.513245,0.149733,1.0,0.0
2,11.25,0.0,0.005970,0.0,0.208178,0.00,0.406780,0.460265,0.256684,1.0,1.0
3,8.63,0.0,0.002985,0.0,0.211896,0.00,0.403390,0.370861,0.267380,1.0,1.0
4,11.92,0.0,0.155224,0.0,0.271375,0.25,0.338983,0.228477,0.454545,1.0,1.0


# preprocessed data ni saqlaymiz

In [47]:
folder = r"C:\Users\bunyo\OneDrive\Desktop\AI_Course\ModularProgramProjects\FirstWeekProject\data\preprocessed_data"
path = os.path.join(folder,'preprocessed_data.csv')
df1.to_csv(path, index=False)